In [1]:
import pandas as pd
import numpy as np
import os

# Process Survey Data

Make sure you download the 2016-2017 Household LSMS survey data for Malawi from https://microdata.worldbank.org/index.php/catalog/lsms. It should be in `../data/input/LSMS/malawi-2016`

This corresponds to ProcessSurveydata.R in the original github

In [2]:
# Preliminaries - 
# define some helper functions that allow us to work with dta files (esp variable labels and value labels) better
def lvar(dta):
    itr = pd.read_stata(dta, iterator = True)
    return itr.variable_labels()
def lcod(dta):
    itr = pd.read_stata(dta, iterator = True)
    return itr.value_labels()    

In [3]:
def join_column(df, fname, column, join_on = 'case_id', convert_cats = True):
    """merges a column or columns from the dta fname onto df"""
    # Error handling needs to convert cats
    newdf= pd.read_stata(fname, convert_categoricals=convert_cats)
    if type(column) == list:
        df = pd.merge(df, newdf[[join_on, *column]], on= join_on)   
    else:
        df = pd.merge(df, newdf[[join_on, column]], on= join_on)
    return df

## Uganda 2011

In [4]:
uga = pd.read_stata('../data/input/LSMS/UGA_2011_UNPS_v01_M_STATA/UNPS 2011-12 Consumption Aggregate.dta')
uga['cons'] = uga['welfare']*118.69/(30*946.89*np.mean([66.68, 71.55]))
fname = '../data/input/LSMS/UGA_2011_UNPS_v01_M_STATA/UNPS_Geovars_1112.dta'
uga = join_column(uga, fname, ['lat_mod','lon_mod','urban'], join_on='HHID')
fname = '../data/input/LSMS/UGA_2011_UNPS_v01_M_STATA/GSEC1.dta'
uga = join_column(uga, fname, ['mult'], join_on='HHID')
fname = '../data/input/LSMS/UGA_2011_UNPS_v01_M_STATA/GSEC9A.dta'
uga = join_column(uga, fname, ['h9q3','h9q4'], join_on='HHID')
uga['metal'] = (uga['h9q4'] == 'Iron sheets')
uga.drop('h9q4', axis =1, inplace = True)
uga.rename(columns = {'lat_mod':'lat','lon_mod':'lon','HHID':'hhid','urban':'rururb','mult':'weight','h9q3':'room'}, inplace = True)
uga

,hhid,round,cpexp30,equiv,welfare,spline,poor,cons,lat,lon,rururb,weight,room,metal
0,1013000201,2011/12,233990.437500,1.738033,134629.437500,29571.796875,0.0,8.138813,-0.530627,32.327492,Rural,1329.515991,2.0,True
1,1013000202,2011/12,59107.175781,2.117033,27919.814453,29571.796875,1.0,1.687849,NaN,NaN,Rural,7844.594727,1.0,True
2,1013000204,2011/12,132707.234375,3.401767,39011.269531,29571.796875,0.0,2.358365,-0.530627,32.327492,Rural,2659.031982,1.0,True
3,1013000206,2011/12,200824.625000,3.001933,66898.429688,29571.796875,0.0,4.044240,0.289081,32.560650,Rural,891.622498,1.0,True
4,1013000210,2011/12,111601.515625,0.992267,112471.296875,29571.796875,0.0,6.799277,-0.530627,32.327492,Rural,1624.854736,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,4193003504,2011/12,103110.281250,5.126600,20112.800781,28165.398438,1.0,1.215888,-0.278874,30.890127,Rural,1476.720947,4.0,True
2812,4193003506,2011/12,411429.406250,6.403733,64248.367188,28165.398438,0.0,3.884035,-0.278874,30.890127,Rural,1437.255859,5.0,False
2813,4193003507,2011/12,72208.257812,2.659733,27148.683594,28165.398438,1.0,1.641231,-0.278874,30.890127,Rural,1334.404175,4.0,True
2814,4193003508,2011/12,109294.093750,3.521900,31032.708984,28165.398438,0.0,1.876034,-0.278874,30.890127,Rural,604.361572,3.0,True


### Tanzania 2011


In [5]:
tza = pd.read_stata('../data/input/LSMS/tanzania_2012/ConsumptionNPS3.dta')
tza['cons'] = tza['expmR']/(365*tza['adulteq'])*112.69/(585.52*np.mean([130.72,141.01]))
fname = '../data/input/LSMS/tanzania_2012/HouseholdGeovars_Y3.dta'
tza = join_column(tza, fname, ['lat_dd_mod','lon_dd_mod'], 'y3_hhid')
fname = '../data/input/LSMS/tanzania_2012/HH_SEC_A.dta'
tza = join_column(tza, fname, ['y3_rural','y3_weight'], 'y3_hhid')
fname = '../data/input/LSMS/tanzania_2012/HH_SEC_I.dta'
tza = join_column(tza, fname, ['hh_i07_1','hh_i09'], 'y3_hhid')
tza['metal'] = tza['hh_i09'] == 'METAL SHEETS (GCI)'
tza.rename(columns = {'lat_dd_mod':'lat','lon_dd_mod':'lon','y3_hhid':'hhid','y3_rural':'rururb','y3_weight':'weight','hh_i07_1':'room'}, inplace = True)
tza.drop('hh_i09', axis = 1, inplace = True)
tza.head()

,hhid,foodbev,alctob,foodIN,foodOUT,utilities,hhexpenses,health,transport,communic,...,area,mainland,quarter,cons,lat,lon,rururb,weight,room,metal
0,0001-001,878800.0,0.0,878800.0,0,59200,123200,656500,0,120000,...,Rural,Mainland,Oct-Dec 2012,2.120255,-5.085751,35.854389,RURAL,5547.124512,2.0,False
1,0002-001,475800.0,0.0,444600.0,31200,31800,3600,0,0,0,...,Rural,Mainland,Oct-Dec 2012,0.770206,-5.085751,35.854389,RURAL,5555.993652,3.0,False
2,0003-001,3536000.0,93600.0,3265600.0,364000,257200,180000,0,336000,180000,...,Rural,Mainland,Oct-Dec 2012,4.027845,-5.085751,35.854389,RURAL,2733.039062,4.0,True
3,0003-010,3445000.0,0.0,2472600.0,972400,602000,268000,303000,322400,300000,...,Dar es Salaam,Mainland,Oct-Dec 2012,5.877266,-6.860888,39.251984,URBAN,1536.331055,2.0,True
4,0005-001,2522375.5,0.0,2418375.5,104000,96600,102800,67600,300000,57600,...,Rural,Mainland,Oct-Dec 2012,3.705670,-5.085751,35.854389,RURAL,5492.644043,3.0,True


## Nigeria 2013

The original code by Jean et al. uses cons_agg_w2.dta.  
THis is no longer in the data. Instead, there are two files: cons_agg_wave2_visit2.dta and cons_agg_wave2_visit1.dta.
Assume that the latter visit was used (i.e. that cons_agg_w2 = cons_agg_wave2_visit2.dta) since they reference one that was post-harvest and that was visit 2 according to the [documentation](https://microdata.worldbank.org/index.php/catalog/1952/study-description)

In [6]:
fname = '../data/input/LSMS/nigeria_2013/cons_agg_wave2_visit2.dta'
nga = pd.read_stata(fname)
#  Assume that totcons = pcexp_dr_w2
nga['totcons'].describe()

count    4.738000e+03
mean     1.254538e+05
std      3.625773e+05
min      6.430951e+03
25%      5.214533e+04
50%      8.058027e+04
75%      1.325398e+05
max      1.842370e+07
Name: totcons, dtype: float64

In [7]:
# Applying the same transformations (presumably for currency and a scaling option and something else)
# gives a distribution for consumption that is roughly in line with the other countries above
# Comparison against tanzania.
nga['cons'] = nga['totcons']/365*110.84/(79.53*100)
print(nga['cons'].describe())
tza['cons'].describe()

count    4738.000000
mean        4.790232
std        13.844375
min         0.245554
25%         1.991077
50%         3.076815
75%         5.060797
max       703.476196
Name: cons, dtype: float64


count    4862.000000
mean        4.558735
std         4.160459
min         0.168748
25%         2.084450
50%         3.333398
75%         5.623975
max        54.232819
Name: cons, dtype: float64

In [8]:
# Now include all the the other files that seem to have remained consistent with when Jean et al. did the paper. 
# seems like they started with nigeria because rururb column name is replicated here
fname = '../data/input/LSMS/nigeria_2013/Geodata Wave 2/NGA_HouseholdGeovars_Y2.dta'
nga = join_column(nga, fname, ['LAT_DD_MOD','LON_DD_MOD','sector'],'hhid', False)
fname = '../data/input/LSMS/nigeria_2013/HHTrack.dta'
nga = join_column(nga, fname, ['wt_wave2'], 'hhid',  False)
fname = '../data/input/LSMS/nigeria_2013/Post Harvest Wave 2/Household/sect8_harvestw2.dta'
nga = join_column(nga, fname, ['s8q9','s8q7'], 'hhid',  False)
nga['metal'] = nga['s8q7'] == 2 # not a string because of the false flags put in read_stata to deal with non-unique column labels.
nga.rename(columns = {'LAT_DD_MOD':'lat','LON_DD_MOD':'lon','wt_wave2':'weight','s8q9':'room'}, inplace = True)
nga.drop('s8q7', axis = 1, inplace = True)
nga.head()

,country,wave,visit,surveyt,surveyprd,zone,state,lga,ea,rururb,...,nfdfoth,totcons,hhweight,cons,lat,lon,sector,weight,room,metal
0,NGA,Second wave,Second - post harvesting,LSMS-ISA2013,2013,4. SOUTH EAST,Abia,115. UMUAHIA NORTH,670,Urban,...,2309.375000,279397.468750,11898.657227,10.668299,5.535456,7.531536,1,11898.656886,5.0,True
1,NGA,Second wave,Second - post harvesting,LSMS-ISA2013,2013,4. SOUTH EAST,Abia,115. UMUAHIA NORTH,670,Urban,...,2700.000000,293407.375000,11898.657227,11.203241,5.535456,7.531536,1,11898.656886,5.0,True
2,NGA,Second wave,Second - post harvesting,LSMS-ISA2013,2013,4. SOUTH EAST,Abia,115. UMUAHIA NORTH,670,Urban,...,4722.222168,194092.812500,11898.657227,7.411090,5.535456,7.531536,1,11898.656886,4.0,True
3,NGA,Second wave,Second - post harvesting,LSMS-ISA2013,2013,4. SOUTH EAST,Abia,115. UMUAHIA NORTH,670,Urban,...,7980.000000,255921.906250,11898.657227,9.771925,5.535456,7.531536,1,11898.656886,4.0,True
4,NGA,Second wave,Second - post harvesting,LSMS-ISA2013,2013,4. SOUTH EAST,Abia,115. UMUAHIA NORTH,670,Urban,...,1091.666626,187675.296875,11898.657227,7.166049,5.535456,7.531536,1,11898.656886,3.0,False


### Malawi 2013/2016

Malawi data doesn't seem to exist by itself - Jean and Burke seem to have either used a subset 203 EAs instead of the 700+ that were part of a time-series survey OR they are using a dataset that is no longer available anymore 

This below uses IHS4 which is was conducted in 2016

In [9]:
# everything in comments I have not converted to Python3 but they don't matter for predicting consumption
# you can play around with the stata files and add additional information to predict if you want
fname = '../data/input/LSMS/malawi_2016/IHS4 Consumption Aggregate.dta'
mwi = pd.read_stata(fname)
# 'Total annual per capita consumption, 
#  spatially & (within IHS4) temporally adjusted' over adult equivalence
mwi['cons'] = mwi['rexpagg']/(365*mwi['adulteq'])
# Not sure where these numbers come from in Jean
mwi['cons'] = mwi['cons']*107.62/(116.28*166.12)

In [10]:
fname = '../data/input/LSMS/malawi_2016/HouseholdGeovariables_stata11/HouseholdGeovariablesIHS4.dta'
mwi = join_column(mwi,fname,['HHID', 'lat_modified', 'lon_modified'])
fname = '../data/input/LSMS/malawi_2016/HH_MOD_A_FILT.dta'
mwi = join_column(mwi,fname,['reside'])
fname = '../data/input/LSMS/malawi_2016/HH_MOD_F.dta'
mwi = join_column(mwi,fname,['hh_f08','hh_f10'])
mwi.rename(columns={'HHID':'hhid','reside': 'rururb','lat_modified': 'lat', 'lon_modified': 'lon','hh_f10': 'room','hh_f08':'metal', 'hh_wgt': 'weight'}, inplace = True)
mwi['metal'] = mwi['metal'] == 'IRON SHEETS'

In [11]:
# just need 6 variables for each
dfs = [mwi,tza, uga, nga]
cs = ['mwi','tza', 'uga', 'nga']
dfs = [df[['cons','rururb','weight','room','metal','lat','lon']].dropna() for df in dfs]
for i in range(len(dfs)):
    dfs[i]['country'] = cs[i]

In [12]:
df = pd.concat(dfs).set_index(['country'])
print(df.shape)
df['urban_encoded'] = df['rururb'].apply(lambda x: x.lower() == 'rural').astype(int)
df.drop('rururb', inplace = True, axis =1)
df

(24717, 7)


,cons,weight,room,metal,lat,lon,urban_encoded
country,,,,,,,
mwi,1.561464,590.332886,2.0,False,-14.683761,34.915074,1
mwi,5.915595,265.032715,3.0,True,-14.005029,33.794591,0
mwi,1.676028,207.084106,3.0,False,-16.826165,35.269503,1
mwi,1.239843,207.147202,3.0,False,-15.004730,35.163219,1
mwi,2.198843,287.574097,2.0,True,-17.016698,35.079629,1
...,...,...,...,...,...,...,...
nga,2.322761,14621.976979,3.0,True,9.277092,7.071132,0
nga,3.759413,14621.976979,5.0,True,9.099142,7.279651,1
nga,1.036274,14621.976979,3.0,True,9.099142,7.279651,1


In [13]:
df = df.dropna()
print(df.shape)
cluster = df.groupby(['country','lon','lat']).mean()
cluster

(24717, 7)


cons       weight      room     metal  \
country lon       lat                                                     
mwi     32.859582 -13.572201  2.212885   300.865601  1.750000  0.187500   
        32.860543 -13.785994  3.715870   246.526001  2.375000  0.687500   
        32.897253 -13.557205  3.209361   456.710602  2.062500  0.375000   
        32.901011 -13.803408  6.832964   284.910614  2.312500  0.812500   
        32.966397 -13.920888  3.419535   460.030792  2.062500  0.125000   
...                                ...          ...       ...       ...   
uga     34.640480  2.566242   1.342921  3133.590096  1.818182  0.090909   
        34.660862  1.331631   2.606161  1388.378348  3.285714  0.714286   
        34.666542  2.541371   7.768633  4430.105469  1.000000  1.000000   
        34.723160  1.845589   3.242811  1556.086957  2.100000  0.100000   
        34.857117  1.769088   1.761717   862.846296  1.428571  0.142857   

                              urban_encoded  
country lon       lat                        
mwi     32.859582 -13.572201            1.0  
        32.860543 -13.785994            1.0  
        32.897253 -13.557205            1.0  
        32.901011 -13.803408            0.0  
        32.966397 -13.920888            1.0  
...                                     ...  
uga     34.640480  2.566242             1.0  
        34.660862  1.331631             1.0  
        34.666542  2.541371             1.0  
        34.723160  1.845589             0.0  
        34.857117  1.769088             1.0  

[3519 rows x 5 columns]

This shows how we can use the .tif file that we downloaded to get nightlight values.

In [14]:
import geoio
filename = '../data/input/Nightlights/2013/F182013.v4c_web.stable_lights.avg_vis.tif'
img = geoio.GeoImage(filename)
xPixel, yPixel = img.proj_to_raster(34.915074, -14.683761)

In [15]:
xPixel, yPixel

(25790.308983159237, 10762.551363048204)

In [16]:
im_array = np.squeeze(img.get_data())

In [17]:
im_array.shape

(16801, 43201)

In [18]:
im_array[int(yPixel),int(xPixel)] # this is how we grab the nightlight values!

0

In [19]:
import math
def create_space(lat, lon):
    # these are pulled from the paper to make the 10km^2 area
    return lat - (180/math.pi)*(5000/6378137), lon - (180/math.pi)*(5000/6378137)/math.cos(lat), \
            lat + (180/math.pi)*(5000/6378137), lon + (180/math.pi)*(5000/6378137)/math.cos(lat)

In [20]:
cluster = cluster.reset_index()
household_nightlights = []
for i,r in cluster.iterrows():
    min_lat, min_lon, max_lat, max_lon = create_space(r.lat, r.lon)
    xminPixel, yminPixel = img.proj_to_raster(min_lon, min_lat)
    xmaxPixel, ymaxPixel = img.proj_to_raster(max_lon, max_lat)
    
    xminPixel, xmaxPixel = min(xminPixel, xmaxPixel), max(xminPixel, xmaxPixel)
    yminPixel, ymaxPixel = min(yminPixel, ymaxPixel), max(yminPixel, ymaxPixel)
    
    xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
    household_nightlights.append(im_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())

In [21]:
os.makedirs('../data/output', exist_ok=True)
os.makedirs('../data/output/LSMS', exist_ok=True)
# os.makedirs('../data/output/LSMS/malawi_2016/', exist_ok=True)

In [22]:
cluster['nightlights'] = household_nightlights
df = pd.merge(df.reset_index(), cluster[['lat', 'lon', 'nightlights']], on=['lat', 'lon'])
cluster = cluster.set_index(['country','lon','lat'])
df = df.reset_index().set_index(['country','lon','lat'])
cluster['num_households'] = df.groupby(level = [0,1,2]).count()['cons']
df['nightlights'] = cluster['nightlights']

In [23]:
# if more than 0.5 average within a clust, label it as 1 (URBAN), else 0
cluster['urban_encoded'] = cluster['urban_encoded'].apply(lambda x: round(x))
# convert back to urban rural coding (non 1-0)
cluster['urban_encoded'] = cluster['urban_encoded'].apply(lambda x: 'RURAL' if x == 0 else 'URBAN')
cluster.rename(columns={'urban_encoded': 'urban'}, inplace=True)

In [24]:
df.to_csv('../data/output/LSMS/all_household.csv', index=True)
cluster.to_csv('../data/output/LSMS/all_cluster.csv', index=True)

In [25]:
# Create npy files
cluster = cluster.reset_index()
np.save('../data/output/LSMS/lats.npy', cluster['lat'].values)
np.save('../data/output/LSMS/lons.npy', cluster['lon'].values)
np.save('../data/output/LSMS/consumptions.npy', cluster['cons'].values)
np.save('../data/output/LSMS/nightlights.npy', cluster['nightlights'].values)
np.save('../data/output/LSMS/households.npy', cluster['num_households'].values)

In [26]:
df

index      cons        weight  room  metal  \
country lon       lat                                                      
mwi     34.915074 -14.683761      0  1.561464    590.332886   2.0  False   
                  -14.683761      1  3.270928    590.332886   1.0  False   
                  -14.683761      2  4.963443    590.332886   2.0   True   
                  -14.683761      3  2.838293    590.332886   1.0   True   
                  -14.683761      4  3.286232    590.332886   1.0  False   
...                             ...       ...           ...   ...    ...   
nga     7.279651   9.099142   24712  3.759413  14621.976979   5.0   True   
                   9.099142   24713  1.036274  14621.976979   3.0   True   
                   9.099142   24714  8.768167  14621.976979  16.0  False   
                   9.099142   24715  1.119193  14621.976979   7.0   True   
        7.071132   9.277092   24716  2.322761  14621.976979   3.0   True   

                              urban_encoded  nightlights  
country lon       lat                                     
mwi     34.915074 -14.683761              1     0.000000  
                  -14.683761              1     0.000000  
                  -14.683761              1     0.000000  
                  -14.683761              1     0.000000  
                  -14.683761              1     0.000000  
...                                     ...          ...  
nga     7.279651   9.099142               1    23.836364  
                   9.099142               1    23.836364  
                   9.099142               1    23.836364  
                   9.099142               1    23.836364  
        7.071132   9.277092               0     0.115702  

[24717 rows x 7 columns]

In [27]:
cluster

,country,lon,lat,cons,weight,room,metal,urban,nightlights,num_households
0,mwi,32.859582,-13.572201,2.212885,300.865601,1.750000,0.187500,URBAN,0.000000,16
1,mwi,32.860543,-13.785994,3.715870,246.526001,2.375000,0.687500,URBAN,1.319648,16
2,mwi,32.897253,-13.557205,3.209361,456.710602,2.062500,0.375000,URBAN,0.000000,16
3,mwi,32.901011,-13.803408,6.832964,284.910614,2.312500,0.812500,RURAL,1.157025,16
4,mwi,32.966397,-13.920888,3.419535,460.030792,2.062500,0.125000,URBAN,0.000000,16
...,...,...,...,...,...,...,...,...,...,...
3514,uga,34.640480,2.566242,1.342921,3133.590096,1.818182,0.090909,URBAN,0.000000,11
3515,uga,34.660862,1.331631,2.606161,1388.378348,3.285714,0.714286,URBAN,0.000000,7
3516,uga,34.666542,2.541371,7.768633,4430.105469,1.000000,1.000000,URBAN,0.000000,1
3517,uga,34.723160,1.845589,3.242811,1556.086957,2.100000,0.100000,RURAL,0.000000,10
